In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  # for plotting
# import seaborn as sns

import scipy.stats as stats  # for statistics

In [ ]:
sheet = pd.read_csv("./Titanic.csv")
sheet.head(5)

0. 男性与女性的年龄分布有无差异
2. 男性与女性的死亡率有无统计学差异
3. 已婚女性与未婚女性死亡率有无统计学差异
4. 死亡率与年龄有无关联

## Q1: 男性与女性的年龄分布有无差异

---


## Q2: 男性与女性的死亡率有无统计学差异

四格表卡方检验
$$ \chi^2 = \frac{n(ad -bc)^2}{(a+b)(b+d)(d+c)(c+a)} $$

## Q3: 已婚女性与未婚女性死亡率有无统计学差异 (optional)

字符串正则表达式

## Q4: 死亡率与年龄有无关联

a glance at plotting